In [15]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from torchaudio import transforms

In [16]:
def qpsk_augmentation(x):
    x = x.clone()

    # 1. Добавляем белый шум (иногда разный по I/Q)
    if torch.rand(1).item() < 0.8:
        noise_level = torch.rand(1).item() * 0.1
        noise = torch.randn_like(x) * noise_level
        x += noise
    
    # 4. Джиттер по времени (мелкие искажения)
    if torch.rand(1).item() < 0.7:
        jitter = torch.randn_like(x) * 0.01
        x += torch.sin(torch.linspace(0, 2 * np.pi, x.shape[1], device=x.device)) * jitter


    

    return x

In [17]:
class QPSKDataset(Dataset):
    def __init__(self, num_samples, time, symbol_rate, sample_rate, carrying_freq, augment_fn):
        self.num_samples = num_samples
        self.time = time
        self.symbol_rate = symbol_rate
        self.sample_rate = sample_rate
        self.carrying_freq = carrying_freq
        self.augment_fn = augment_fn
        self.samples_per_symbol = int(sample_rate / symbol_rate)

    def __len__(self):
        return self.num_samples

    def generate_qpsk(self):
        num_symbols = int(self.time * self.symbol_rate)
        num_samples = int(self.time * self.sample_rate)
        samples_per_symbol = int(self.sample_rate / self.symbol_rate)

        bits = np.random.randint(0, 2, size=num_symbols * 2)
        symbols = bits.reshape(-1, 2)

        phase_map = {
            (0, 0): (1, 1),
            (0, 1): (-1, 1),
            (1, 1): (-1, -1),
            (1, 0): (1, -1)
        }

        iq = np.array([phase_map[tuple(b)] for b in symbols])
        i_vals, q_vals = iq[:, 0], iq[:, 1]

        i_samples = np.repeat(i_vals, samples_per_symbol)
        q_samples = np.repeat(q_vals, samples_per_symbol)

        t = np.linspace(0, self.time, int(self.time * self.sample_rate), endpoint=False)

        carrier_cos = np.cos(2 * np.pi * self.carrying_freq * t) * (np.sqrt(2)/2)
        carrier_sin = np.sin(2 * np.pi * self.carrying_freq * t) * (np.sqrt(2)/2)

        signal = i_samples * carrier_cos - q_samples * carrier_sin
        iq_signal = np.stack([i_samples*carrier_cos, q_samples*carrier_sin], axis=0)  # Shape: [2, T]
        return signal.astype(np.float32)

    def __getitem__(self, idx):
        clean = self.generate_qpsk()
        noisy = self.augment_fn(torch.tensor(clean.copy())) if self.augment_fn else clean
        return noisy, clean  # (зашумленный, чистый)



In [18]:

class QPSKDataModule(pl.LightningDataModule):
    def __init__(self, batch_size=32, num_workers=0, **signal_params):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.signal_params = signal_params
        self.samples_per_symbol = int(signal_params["sample_rate"] / signal_params["symbol_rate"])

    def setup(self, stage=None):
        self.dataset = QPSKDataset(
            num_samples=10000,
            augment_fn=qpsk_augmentation,
            **self.signal_params
        )
        self.val_dataset = QPSKDataset(
            num_samples=1000,
            augment_fn=None,
            **self.signal_params
        )

        self.test_dataset =QPSKDataset(
            num_samples=1000,
            augment_fn=qpsk_augmentation,
            **self.signal_params
        )

    def train_dataloader(self):
        return DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
            drop_last=True,
        )
    
    def val_dataloader(self):
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size, 
            shuffle=False, 
            num_workers=self.num_workers
        )
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, 
            batch_size=self.batch_size, 
            shuffle=False, 
            num_workers=self.num_workers
        )


    def get_samples_per_symbol(self):
        return self.samples_per_symbol

In [19]:
class DNN_PNC(nn.Module):
    def __init__(self, input_channels=1, num_filters=64):
        super(DNN_PNC, self).__init__()
        
        # Энкодер
        self.encoder1 = self._block(input_channels, num_filters)
        self.encoder2 = self._block(num_filters, num_filters*2)
        self.encoder3 = self._block(num_filters*2, num_filters*4)
        self.encoder4 = self._block(num_filters*4, num_filters*8)
        
        # Боттлнек
        self.bottleneck = self._block(num_filters*8, num_filters*16)
        
        # Декодер
        self.decoder1 = self._block(num_filters*16, num_filters*8)
        self.decoder2 = self._block(num_filters*8, num_filters*4)
        self.decoder3 = self._block(num_filters*4, num_filters*2)
        self.decoder4 = self._block(num_filters*2, num_filters)
        
        # Выходной слой
        self.out_conv = nn.Conv1d(num_filters, input_channels, kernel_size=1)
        
        # Пулинг и апсемплинг
        self.pool = nn.MaxPool1d(2)
        self.up = nn.Upsample(scale_factor=2, mode='linear', align_corners=True)
        
    def _block(self, in_channels, out_channels):
        return nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        if x.dim() == 2:
            x = x.unsqueeze(1)
        # Энкодер
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool(enc1))
        enc3 = self.encoder3(self.pool(enc2))
        enc4 = self.encoder4(self.pool(enc3))
        
        # Боттлнек
        bottleneck = self.bottleneck(self.pool(enc4))
        
        # Декодер
        dec1 = self.up(bottleneck)
        dec1 = torch.cat([dec1, enc4], dim=1)
        dec1 = self.decoder1(dec1)
        
        dec2 = self.up(dec1)
        dec2 = torch.cat([dec2, enc3], dim=1)
        dec2 = self.decoder2(dec2)
        
        dec3 = self.up(dec2)
        dec3 = torch.cat([dec3, enc2], dim=1)
        dec3 = self.decoder3(dec3)
        
        dec4 = self.up(dec3)
        dec4 = torch.cat([dec4, enc1], dim=1)
        dec4 = self.decoder4(dec4)
        
        # Выход
        out = self.out_conv(dec4)
        return out

In [20]:
class QPSKDenoiser(pl.LightningModule):
    def __init__(self, model, lr=1e-3):
        super().__init__()
        self.model = model
        self.lr = lr
        self.loss_fn = nn.MSELoss()
        
    def forward(self, x):

        if x.dim() == 2:
            x = x.unsqueeze(1)
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        noisy, clean = batch
        predicted_noise = self(noisy)
        denoised = noisy - predicted_noise.squeeze(1)
        loss = self.loss_fn(denoised, clean)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        noisy, clean = batch
        predicted_noise = self(noisy)
        denoised = noisy - predicted_noise.squeeze(1)
        loss = self.loss_fn(denoised, clean)
        self.log("val_loss", loss)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr)

In [21]:
# Параметры сигнала
signal_params = {
    "time": 5.0,           # длительность сигнала в секундах
    "symbol_rate": 6,   # скорость символов (бод)
    "sample_rate": 120 ,  # частота дискретизации
    "carrying_freq": 12.0  # несущая частота
}

# Инициализация
model = DNN_PNC(input_channels=1)
logger = TensorBoardLogger("tb_logs", name="qpsk_denoiserwithunet")
datamodule = QPSKDataModule(batch_size=32, **signal_params)
trainer = Trainer(
    max_epochs=1001,
    logger=logger,
    accelerator="auto",
    devices=1 if torch.cuda.is_available() else None,
    log_every_n_steps=10,
    check_val_every_n_epoch=1
)
denoiser = QPSKDenoiser(model)

# Обучение
trainer.fit(denoiser, datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | model   | DNN_PNC | 9.4 M  | train
1 | loss_fn | MSELoss | 0      | train
--------------------------------------------
9.4 M     Trainable params
0         Non-trainable params
9.4 M     Total params
37.722    Total estimated model params size (MB)
68        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 74 but got size 75 for tensor number 1 in the list.

In [ ]:
def denoise_signal(model, noisy_signal):
    with torch.no_grad():
        noisy_tensor = torch.tensor(noisy_signal).unsqueeze(0).unsqueeze(0).float()
        predicted_noise = model(noisy_tensor)
        denoised = noisy_tensor - predicted_noise
    return denoised.squeeze().numpy()